In [1]:
import pandas as pd
df = pd.read_csv('events.csv')

from datetime import datetime
df['date'] = df['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000))

# Diviser entre transactions et le reste
non_transactions = df.loc[df.event != "transaction"]
transactions = df.loc[df.event == "transaction"]

In [3]:
# Créer un dataframe par merge inner
inner_merged_total = pd.merge(transactions, non_transactions, on=["visitorid"], how="inner")

# Calculez la différence entre l'heure à laquelle la transaction s'est produite
# et l'heure à laquelle chaque événement s'est produit
inner_merged_total['time_diff'] = inner_merged_total['date_x'] - inner_merged_total['date_y']

# Trier le dataframe par visitorid...
inner_merged_total = inner_merged_total.sort_values(by=['visitorid','date_x','itemid_x','time_diff'])


In [5]:
import numpy as np

# Appliquer 2 conditions importantes pour définir une visite convertissante : 
# Premièrement, time_diff est moins de 24h. En deuxième lieu, le itemid sur lequel la transaction 
# a eu lieu et le itemid sur lequel chaque événement s'est produit doivent être les mêmes
transaction_24h = inner_merged_total.loc[inner_merged_total['time_diff'] < np.timedelta64(1, 'D')]
transaction_made_24h = transaction_24h.loc[inner_merged_total['itemid_x'] == inner_merged_total['itemid_y']]

# Supprimer certaines colonnes
transaction_made_24h = transaction_made_24h.drop(['timestamp_x','timestamp_y','transactionid_y'], axis=1)

# Ajouter une colonne 'converting' qui peut contenir les valeurs OUI(=1) ou NON(=0 PAR DEFAUT)
transaction_made_24h['converting_time_diff'] = transaction_made_24h['time_diff'].diff()
transaction_made_24h['converting'] = 0

# Afin d'alimenter la colonne 'converting', appliquer un autre time diff sur la colonne 'time_diff' et 
# enregistrer le résultat dans la colonne 'converting_time_diff'
# Si le résultat est négatif, assigner la valeur = 1, c'est à dire que 
# c'est une nouvelle transaction (autrement dit une visite convertissante).
transaction_made_24h.loc[transaction_made_24h['converting_time_diff'] < np.timedelta64(0, 'D'),'converting'] = 1
transaction_made_24h.iloc[0, transaction_made_24h.columns.get_loc('converting')] = 1
transaction_made_24h

# A FAIRE CE SOIR : CALCULER LE NOMBRE DE CONVERTING = 1

,visitorid,event_x,itemid_x,transactionid_x,date_x,event_y,itemid_y,date_y,time_diff,converting_time_diff,converting
10069106,172,transaction,465522,9725.0,2015-08-15 03:29:01.230,view,465522,2015-08-15 03:27:27.421,0 days 00:01:33.809000,NaT,1
10069098,172,transaction,465522,9725.0,2015-08-15 03:29:01.230,view,465522,2015-08-15 03:27:09.037,0 days 00:01:52.193000,0 days 00:00:18.384000,0
10069105,172,transaction,465522,9725.0,2015-08-15 03:29:01.230,addtocart,465522,2015-08-15 03:13:48.324,0 days 00:15:12.906000,0 days 00:13:20.713000,0
10069108,172,transaction,465522,9725.0,2015-08-15 03:29:01.230,addtocart,465522,2015-08-15 03:13:39.691,0 days 00:15:21.539000,0 days 00:00:08.633000,0
10069099,172,transaction,465522,9725.0,2015-08-15 03:29:01.230,view,465522,2015-08-15 03:06:38.438,0 days 00:22:22.792000,0 days 00:07:01.253000,0
...,...,...,...,...,...,...,...,...,...,...,...
9743949,1407110,transaction,360922,3851.0,2015-08-05 01:11:25.624,view,360922,2015-08-05 00:57:13.885,0 days 00:14:11.739000,0 days 00:03:12.976000,0
9743947,1407110,transaction,360922,3851.0,2015-08-05 01:11:25.624,view,360922,2015-08-05 00:56:24.880,0 days 00:15:00.744000,0 days 00:00:49.005000,0
9743944,1407110,transaction,360922,3851.0,2015-08-05 01:11:25.624,view,360922,2015-08-05 00:52:55.493,0 days 00:18:30.131000,0 days 00:03:29.387000,0
9743948,1407110,transaction,360922,3851.0,2015-08-05 01:11:25.624,view,360922,2015-08-05 00:51:55.493,0 days 00:19:30.131000,0 days 00:01:00,0
